# Importe as bibliotecas necessárias

In [ ]:
'''
# Instalando as bibliotecas necessárias
%pip install -U pip setuptools wheel
%pip install pymongo
%pip install pandas
%pip install nltk
%pip install wordcloud
%pip install spacy
%pip install matplotlib
%pip install numpy==1.26.4
%pip install -U scikit-learn
%pip install unidecode
'''

# Importando as bibliotecas

In [ ]:
import pandas as pd
import nltk

nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
nltk.download('averaged_perceptron_tagger_eng', quiet=True)
from datetime import datetime
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from db import connection_db as conndb
from db import filters
from utils import plot_graphs
from utils.text_treatment import TextTreatment
from dateutil.relativedelta import relativedelta

In [ ]:
mongo_connection = conndb.MongoDBConnection(uri='mongodb://localhost:27017/', database_name='dadosVivamente',
                                            collection_name='dadosSemFiltros')
mongo_connection.connect()
collection = mongo_connection.collection

In [ ]:
collection_filters = filters.CollectionFilters(collection)
collection_filters.apply_pipeline1('dadosComFiltrosIniciais')
collection_filters.apply_pipeline2(7, 2, 'posts7anos2anos')
collection_filters.apply_pipeline3('postsComBDIAndInfos')
collection_filters.apply_pipeline4('postsComBDIAndInfosFiltroDataPosts')
data_inicio = datetime(2017, 12, 1)
data_fim = data_inicio - relativedelta(months=6)
collection_filters.apply_pipeline5('postsFiltradosPorData', data_inicio, data_fim)
collection_filters.quant_users_cat('suicida', '$eq', '3')
collection_filters.count_users_by_gender('suicida', '$eq', '3', 'M')
collection_filters.count_users_by_gender('suicida', '$eq', '3', 'F')
collection = collection_filters.collection

In [ ]:
# Filtrando documentos com o atributo maior que 3
filtro = {"suicida": {"$eq": "3"}}
documentos = collection.find(filtro)

In [ ]:
# Transformando os documentos em um DataFrame
df = pd.DataFrame(list(documentos))
df.head()

In [ ]:
print(df.dtypes)
# Lista de colunas que precisam ser convertidas
colunas_para_converter = [
    'pessimismo', 'tristeza', 'fracasso', 'prazer', 'culpa', 'punicao', 'estima',
    'critica', 'suicida', 'choro', 'agitacao', 'interesse', 'indecisao',
    'desvalorizacao', 'energia', 'sono', 'irritabilidade', 'apetite',
    'concentracao', 'fadiga', 'int_sexo', 'quantAmigos'
]

df[colunas_para_converter] = df[colunas_para_converter].astype('int64')

In [ ]:
# Salvar o DataFrame em um arquivo CSV
df.to_csv('dados/com_filtros_datas/6meses/so_suicida_6_meses.csv', index=False)

In [ ]:
# Agrupar por usuário, mês e ano
posts_grouped = df.groupby(['id_usuario', 'mes', 'ano']).size().reset_index(name='quantidade')

# Adicionar coluna com o período e converter para datetime
posts_grouped['periodo'] = pd.to_datetime(posts_grouped['mes'].astype(str) + '/' + posts_grouped['ano'].astype(str),
                                          format='%m/%Y')

# Deve retornar 0 se a conversão foi bem-sucedida.
print(posts_grouped['periodo'].isnull().sum())

posts_grouped.dtypes

# Plotar quantidade de posts por usuário

In [ ]:
plot_graphs.PlotGraphs().plot_posts_per_user(posts_grouped, 'dados/com_filtros_datas/6meses/graficos')

# Efentuando a limpeza dos dados

In [ ]:
df['postMessageLimpo'] = df['postMessage'].fillna('').apply(lambda text: TextTreatment(text).preprocessamento_texto())

In [ ]:
# Exibir apenas as colunas postMessage e postMessageLimpo
df[['postMessage', 'postMessageLimpo']].head()

In [ ]:
# def get_stopwords() -> list:
#     portuguese_ingles_stopwords = []
# 
#     # Stopwords em português
#     portugues = [unidecode.unidecode(palavra.lower().replace(" ", "")) for palavra in
#                  nltk.corpus.stopwords.words('portuguese')]
#     portuguese_ingles_stopwords.extend(portugues)
# 
#     # Stopwords em inglês
#     ingles = [unidecode.unidecode(palavra.lower().replace(" ", "")) for palavra in
#               nltk.corpus.stopwords.words('english')]
#     portuguese_ingles_stopwords.extend(ingles)
# 
#     # Stopwords de diferentes fontes
#     stopwords_wordcloud = [unidecode.unidecode(palavra.lower().replace(" ", "")) for palavra in STOPWORDS]
#     portuguese_ingles_stopwords.extend(stopwords_wordcloud)
# 
#     stopwords_sklearn = [unidecode.unidecode(word.lower().replace(" ", "")) for word in STOP_WORDS]
#     portuguese_ingles_stopwords.extend(stopwords_sklearn)
# 
#     # Adicionar stopwords do arquivo customizado
#     with open('dados/datasets/stopwords-pt.txt', 'r', encoding='utf-8') as words:
#         custom_stopwords = [unidecode.unidecode(word.lower().strip()) for word in words if word.strip()]
#     portuguese_ingles_stopwords.extend(custom_stopwords)
# 
#     # Adicionar nomes
#     with open('dados/datasets/nomes.txt', 'r', encoding='utf-8') as words:
#         nomes_stopwords = [unidecode.unidecode(word.lower().strip()) for word in words if word.strip()]
#     portuguese_ingles_stopwords.extend(nomes_stopwords)
# 
#     # Remover duplicatas e ordenar a lista
#     portuguese_ingles_stopwords = list(set(portuguese_ingles_stopwords))
#     portuguese_ingles_stopwords.sort()
# 
#     return portuguese_ingles_stopwords
# 
# 
# def remocao_stopword(string, lista_stopwords) -> str:
#     a = [i for i in string.split() if i not in lista_stopwords]
#     return ' '.join(a)
# 
# 
# def remove_caracteres(text) -> str:
#     text = text.lower().strip()
#     text = re.compile(r'<.*?>').sub('', text)
#     text = re.compile(r'[%s]' % re.escape(string.punctuation)).sub(' ', text)
#     text = re.sub(r'\s+', ' ', text)
#     text = re.sub(r'\[[0-9]*\]', ' ', text)
#     text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
#     text = re.sub(r'\d', ' ', text)
#     text = re.sub(r"\$", "", text)
#     text = re.sub(r"https?:\/\/.*[\r\n]*", "", text)
#     text = re.sub(r"#", "", text)
#     text = re.sub(r'https?:\/\/[\r\n],"[\r\n]"', '', text, flags=re.MULTILINE)
#     text = re.sub(r'\<a href', ' ', text)
#     text = re.sub(r'&amp;', '', text)
#     text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
#     text = re.sub(r'<br />', ' ', text)
#     text = re.sub(r'\'', ' ', text)
#     text = re.sub(r'[^a-zà-ù ]', ' ', text)
#     text = re.sub(r'k{2,}', '', text, flags=re.IGNORECASE)
#     text = re.sub(r'j{2,}', '', text, flags=re.IGNORECASE)
#     text = re.compile(r"["
#                       u"\U0001F600-\U0001F64F"  # emoticons
#                       u"\U0001F300-\U0001F5FF"  # symbols & pictographs
#                       u"\U0001F680-\U0001F6FF"  # transport & map symbols
#                       u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
#                       u"\U00002500-\U00002BEF"  # chinese char
#                       u"\U00002702-\U000027B0"
#                       u"\U00002702-\U000027B0"
#                       u"\U000024C2-\U0001F251"
#                       u"\U0001f926-\U0001f937"
#                       u"\U00010000-\U0010ffff"
#                       u"\u2640-\u2642"
#                       u"\u2600-\u2B55"
#                       u"\u200d"
#                       u"\u23cf"
#                       u"\u23e9"
#                       u"\u231a"
#                       u"\ufe0f"  # dingbats
#                       u"\u3030"
#                       "]+", flags=re.UNICODE).sub(r'', text)
#     return text
# 
# 
# def obter_pos_tag(token) -> str:
#     if token.startswith('J'):
#         return wordnet.ADJ
#     elif token.startswith('V'):
#         return wordnet.VERB
#     elif token.startswith('N'):
#         return wordnet.NOUN
#     elif token.startswith('R'):
#         return wordnet.ADV
#     else:
#         return wordnet.NOUN
# 
# 
# def lematizacao(string) -> str:
#     token = word_tokenize(string)
#     word_pos_tags = nltk.pos_tag(token)
#     wl = WordNetLemmatizer()
#     a = [wl.lemmatize(tag[0], obter_pos_tag(tag[1])) for idx, tag in
#          enumerate(word_pos_tags)]
#     return " ".join(a)
# 
# 
# def preprocessamento_texto(texto_limpo) -> str:
#     lista_stopwords = get_stopwords()
#     texto_limpo = remove_caracteres(texto_limpo)
#     texto_limpo = remocao_stopword(texto_limpo, lista_stopwords)
#     texto_limpo = lematizacao(texto_limpo)
#     return texto_limpo

In [ ]:
corpus = df['postMessageLimpo'].tolist()

In [ ]:
pd.set_option('display.max_colwidth', None)
df.head()

In [ ]:
# Carregar stopwords em português para os TF-IDF e Bag of Words
from nltk.corpus import stopwords

stop_words = stopwords.words('portuguese')

In [ ]:
# Criar o vetorizador TF-IDF com parâmetros ajustados
vectorizer = TfidfVectorizer(
    stop_words=stop_words,
    max_features=None,
    max_df=0.40,
    min_df=5,
    ngram_range=(1, 1),
    sublinear_tf=True
)

corpus_tfidf = df['postMessageLimpo'].tolist()
tfidf_matrix = vectorizer.fit_transform(corpus_tfidf)

# Obter as palavras
palavras_tfidf = vectorizer.get_feature_names_out()
# Lista para armazenar os resultados
resultados_tfidf = []

# Iterar sobre cada usuário
for usuario in df['id_usuario'].unique():
    # Filtrar os textos do usuário
    indices_usuario = df[df['id_usuario'] == usuario].index
    if len(indices_usuario) > 0:
        # Extrair as palavras com maior score TF-IDF para o usuário
        user_tfidf = tfidf_matrix[indices_usuario]
        user_tfidf_mean = np.asarray(user_tfidf.mean(axis=0)).flatten()

        # Obter os índices das 10 palavras com maior score
        top_10_indices = user_tfidf_mean.argsort()[-10:][::-1]

        # Adicionar as palavras e seus scores à lista de resultados
        for index in top_10_indices:
            resultados_tfidf.append({
                'id_usuario': usuario,
                'palavra': palavras_tfidf[index],
                'score': user_tfidf_mean[index]
            })

# Converter a lista de resultados em DataFrame
resultados_df_tfidf = pd.DataFrame(resultados_tfidf)
resultados_df_tfidf.to_csv('dados/com_filtros_datas/6meses/so_suicida_resultados_tfidf_unigramas.csv', index=False)

print(resultados_df_tfidf)

Usando o algoritmo Bag of Words

In [ ]:
# Criar o vetorizador Bag of Words com parâmetros ajustados
vectorizer = CountVectorizer(
    stop_words=stop_words,
    ngram_range=(1, 1),
    max_df=0.85,
    min_df=5
)

corpus_bow = df['postMessageLimpo'].tolist()
bow_matrix = vectorizer.fit_transform(corpus_bow)

palavras_bow = vectorizer.get_feature_names_out()

resultados_bow = []

for usuario in df['id_usuario'].unique():
    indices_usuario = df[df['id_usuario'] == usuario].index
    if len(indices_usuario) > 0:
        user_bow = bow_matrix[indices_usuario]
        user_bow_sum = np.asarray(user_bow.sum(axis=0)).flatten()

        top_10_indices_bow = user_bow_sum.argsort()[-10:][::-1]

        for index in top_10_indices_bow:
            resultados_bow.append({
                'id_usuario': usuario,
                'palavra': palavras_bow[index],
                'contagem': user_bow_sum[index]
            })

# Converter a lista de resultados em DataFrame
resultados_df_bow = pd.DataFrame(resultados_bow)
resultados_df_bow.to_csv('dados/com_filtros_datas/6meses/so_suicida_resultados_bow_unigramas.csv', index=False)

print(resultados_df_bow)